[Table of Contents](../../index.ipynb)

# FRC Analytics with Python - Session 18
# Structured Query Language (SQL) - Part II
**Last Updated: 8 November 2021**

In SQL Part I we used SQL *SELECT* queries to retrieve information from database tables. In this notebook, we'll retrieve information from multiple tables at one time. We'll also learn how to edit database records and insert new records into the database. Finally, we'll learn how to create new database tables.

## I. Notebook Setup

### A. If Using Google Colab
It's best if you clone the *pyclass_frc* Github repo and run this notebook from your local computer. But if you would like to run it from Google Colab, uncomment and run the line in the next cell. (*Don't delete the exclamation point at the start of the line!*) The cell will copy a Sqlite database file from the Github repository.

In [ ]:
# !wget -nv https://raw.githubusercontent.com/irs1318dev/pyclass_frc/master/sessions/s18_SQL_II/wasno2020edited.sqlite3
# !wget -nv https://raw.githubusercontent.com/irs1318dev/pyclass_frc/master/sessions/s18_SQL_II/chinook.sqlite3

### B. Imports and Database File
Run the next cell to set up the notebook to work with our SQLite database.

In [ ]:
import sqlite3
import sys

import pandas as pd

# Database files
wasno_db = "wasno2020edited.sqlite3"
chinook_db = "chinook.sqlite3"

### C. SQL References
For your convenience, here are the SQL references that were discussed in session 17.
* [Official SQLite documentation](https://www.sqlite.org/lang.html)
* [Python sqlite3 Package Documentation](https://docs.python.org/3/library/sqlite3.html)
* [W3 Schools SQL Tutorial](https://www.w3schools.com/Sql/default.asp) 

## II. Joining Tables
Let's look at the first dozen rows in the *schedule* table.

In [ ]:
query = """SELECT *
             FROM schedule
            LIMIT 12;"""
con = sqlite3.connect(wasno_db)
sched = pd.read_sql_query(query, con, index_col="match_id")
con.close()
sched

The schedule table lists all qualification matches for the robotics competition. There are six teams in every match, so there are six records in the *schedule* table for every match. 

### A. LEFT JOIN
The output from the previous query contains the team number, but not the team's name. Let's fix that.

In [ ]:
query = """SELECT schedule.match_desc, schedule.alliance,
                  schedule.team, schedule.station, teams.team_name
             FROM schedule LEFT JOIN teams
               ON schedule.team = teams.team_number;"""
con = sqlite3.connect(wasno_db)
sched = pd.read_sql_query(query, con, index_col="match_desc")
con.close()
sched.head(6)

The preceding query contains a `LEFT JOIN` clause.
* For a `LEFT JOIN`, SQL creates a record for every row in the *schedule* table. All rows are returned from the *schedule* table because it is on the *LEFT* of the `LEFT JOIN` phrase. Get it? 
* Next, for every row in the *schedules* table, SQL finds a row in the teams table where *teams.team_number* has the same value as *schedule.team*, as specified in the `ON` clause.
* Finally, SQL adds the corresponding value of *teams.team_name* to the record and returns the results.

Note how we prefaced the column selectors with the name of the table: `schedule.match_id`, `teams.team_name`, etc. This is not always necessary. If a column exists in only one of the tables, SQL can usually figure out what table it's in even if the table name is unspecified. But if a column exists in both tables, SQL will throw an error if the table name is not specified. It's a good practice to always specify the table name. The query will always work, even if later on someone adds a new, duplicate column to one of the tables.

### B. INNER JOIN
SQLite also supports the `INNER JOIN` clause. Inner joins behave differently than left joins. Run the next cell to investigate.

In [ ]:
# Invsestigate differences between left and right joins

# How many qualification matches were there?
con = sqlite3.connect(wasno_db)
num_matches_query = """SELECT COUNT(DISTINCT match_desc)
                         FROM schedule;"""
num_matches = con.execute(num_matches_query).fetchone()[0]
print("Number of matches at competition:\t", num_matches)
print("\nNumber of teams in each match:\t\t", 6)

# How many rows do we expect in our schedule table?
print("\nExpected number of rows in\nschedule table (6 x 60):\t\t",
      6 * num_matches)

# How many rows were returned from the LEFT JOIN?
left_join_query = """SELECT COUNT(*)
                       FROM schedule LEFT JOIN teams
                         ON schedule.team = teams.team_number;"""
print("\nNumber of records for LEFT JOIN:\t",
      con.execute(left_join_query).fetchone()[0])

# How many rows were returned from the INNER JOIN?
inner_join_query = """SELECT COUNT(*)
                        FROM teams INNER JOIN schedule
                          ON schedule.team = teams.team_number;"""
print("\nNumber of records for INNER JOIN:\t",
      con.execute(inner_join_query).fetchone()[0])

con.close()

You might have noticed that we're using different functions to run the queries. We'll explain that later. The important thing is that while the `LEFT JOIN` returned all 360 records from the *schedule* table (as expected), the `INNER JOIN` dropped 29 records. Why is that?

In [ ]:
# Investigate Differences in number of records
con = sqlite3.connect(wasno_db)
team_query = """SELECT COUNT(DISTINCT team_number) FROM teams;"""
print("\nNumber of teams in teams table:\t\t",
      con.execute(team_query).fetchone()[0])

sched_query = """SELECT COUNT(DISTINCT team) FROM schedule;"""
print("\nNumber of teams in schedule table:\t",
      con.execute(sched_query).fetchone()[0])
con.close()

The different joins returned different results because there are three teams missing from the *teams* table. (The mentor deleted three teams to demonstrate the difference between inner and left joins.)
* The `INNER JOIN` will only return records when both the *teams* and *schedule* table have rows with matching values in the *teams.team_number* and *schedule.team* fields. There are 29 rows in the schedule table for which there are no corresponding rows in the teams table, so these 29 rows are not returned.
* In contrast, the `LEFT JOIN` returns all rows from the *schedule* table, even if there is no corresponding row in the *teams* table.

See Figure 1 for an illustration of how the two different types of joins work.

#### Figure 1, Join Type Comparison
![Join Diagram](images/join_figure.png)

You can also read the explanations in the *W3 Schools SQL Tutorial* for additional explations.
* [Inner Joins](https://www.w3schools.com/Sql/sql_join_inner.asp)
* [Left Joins](https://www.w3schools.com/Sql/sql_join_left.asp)

### C. RIGHT JOIN and FULL OUTER JOIN
Most SQL databases support a `RIGHT JOIN.` A `RIGHT JOIN` returns all records from the table on the right side of the `JOIN` clause, but only records with matching values from the table on the left side. SQLite does NOT support right joins. This is not a problem because a left join can be used in place of a right join if we just swap the two tables in the join clause. 

Many SQL databases support a `FULL OUTER JOIN`, which returns all records from both the left-side and right-side tables. SQLite does not support full outer joins either, but there are workarounds.

Here are more relevant sections from the *W3 Schools SQL Tutorial*.
* [Join Overview](https://www.w3schools.com/Sql/sql_join.asp)
* [Right Joins (NOT SUPPORTED BY SQLite)](https://www.w3schools.com/Sql/sql_join_right.asp)
* [Full Joins (NOT SUPPORTED BY SQLite)](https://www.w3schools.com/Sql/sql_join_full.asp)

### D. Cross Joins (Cartesian Products)
We demonstrated in section B that the inner join of the *Schedule* and *Teams* table returned 360 rows. What happens if we leave off the `ON` clause?

In [ ]:
query = """SELECT COUNT(*) AS num_rows
             FROM schedule LEFT JOIN teams;"""
con = sqlite3.connect(wasno_db)
sched = pd.read_sql_query(query, con)
con.close()
sched.head(6)

Really? 12,240 rows? What's happening here? To understand the answer, keep in mind that there are 34 rows in the *Teams* table and 360 rows in the *Schedule* table.
$$ 34 \times 360 = 12,240$$
When the `ON` clause is left out, SQL combines every row with every other row. The following figure shows what would happen if we combined two three-row tables without an `ON` clause.

#### Figure 2, Cross Joins
![Cartesian Product](images/cartesian_product.png)

Cross joins don't occur very often, at least not on purpose. Knowing about cross joins is helpful for troubleshooting misbehaving queries. If your query is unexpectedly returning a crazy amount of records, check your join condition. You might have inadvertently created a cross join.

[The term *Cartesian product* comes from set theory.](https://en.wikipedia.org/wiki/Cartesian_product) Cartesian products are named after René Descartes, who founded analytic geometry in the 1600s. If we have two sets, $A$ and $B$, the Cartesian product $A \times B$ is the set of all ordered pairs, where the first element is from $A$ and the second element is from $B$. In set notation:

$$ A \times B = \{(a, b) \;|\; a \in A \; \textrm{and} \; b \in B\}$$

The symbol $\in$ means *is an element of*. For example, [Emmanuelle Charpentier](https://en.wikipedia.org/wiki/Emmanuelle_Charpentier) and [Jennifer Doudna](https://en.wikipedia.org/wiki/Jennifer_Doudna) are elements of the set of Nobel prize winners in chemistry.

### E. Old Fashioned `WHERE` Joins
An inner join can be accomplished by using `WHERE` instead of `ON`, and without the `INNER JOIN` clause. Consider the following query.

In [ ]:
query = """SELECT schedule.match_desc, schedule.alliance,
                  schedule.team, schedule.station, teams.team_name
             FROM schedule, teams
            WHERE schedule.team = teams.team_number;"""
con = sqlite3.connect(wasno_db)
sched = pd.read_sql_query(query, con, index_col="match_desc")
con.close()
sched.head(6)

This query works by creating a Cartesian product, and then selecting only the records where the team numbers match. This technique works, but it's considered better form to use `JOIN` and `ON` clauses.

### F. Multiple Table Joins
We're going to use a different database for the next section. The *chinook.sqlite3* database contains sales information for a fictional digital music store in Alberta, Canada. [It is available for several different database servers and can be downloaded from Github.](https://github.com/lerocha/chinook-database)

The chinook database has eleven tables, as shown below.

In [ ]:
query = """SELECT *
             FROM sqlite_schema
            WHERE type = 'table';"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df

There is a playlist table. That sounds cool. Let's check it out.

In [ ]:
query = """SELECT *
             FROM Playlist;"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df

Hmmm. The *Playlist* table only contains the playlist names. We would like to see what songs are on each playlist. Perhaps there are some clues in the *PlaylistTrack* table?

In [ ]:
query = """SELECT *
             FROM PlaylistTrack;"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df.head()

Both the *Playlist* and the *PlaylistTrack* tables contain a column named *PlaylistId*. What would happen if we were to join the two tables on that column? To keep the results manageable, let's limit our results to the Grunge playlist.

In [ ]:
query = """SELECT Playlist.Name, PlaylistTrack.TrackId
             FROM Playlist INNER JOIN PlaylistTrack
               ON Playlist.PlaylistId = PlaylistTrack.PlaylistId
            WHERE Playlist.Name = 'Grunge';"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df

I looks like the Grunge playlist has 15 tracks, but we don't know what the tracks are. All we have are random-looking *TrackId* numbers. Let's check out the *Track* table to see if that has some hints.

In [ ]:
query = """SELECT *
             FROM Track;"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df.head()

The *Track* table also has a *TrackId* column. Can we join the *Track* table to the *Playlist* and *PlaylistTrack* tables? Yes, we can.

In [ ]:
query = """SELECT Playlist.Name AS Playlist, PlaylistTrack.TrackId,
                  Track.Name AS Track, Track.Composer AS Artist
            FROM Playlist
                 INNER JOIN PlaylistTrack
                         ON Playlist.PlaylistId = PlaylistTrack.PlaylistId
                  LEFT JOIN Track
                         ON PlaylistTrack.TrackId = Track.TrackId
           WHERE Playlist.Name = 'Grunge';"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con, index_col="TrackId")
con.close()
df

We finally have results that make sense, at least to humans. Let's break down the query.
* We added an additional table to our join by adding a second join clause. In this example, the second join was a left join.
* Imagine that each `JOIN` statment is creating an intermediate table. The first statement, `Playlist INNER JOIN PlaylistTrack ON Playlist.PlaylistId = PlaylistTrack.PlaylistId` creates an intermediate table with the *Playist.Name* and *PlaylistTrack.TrackId* columns. The second join statement, `... LEFT JOIN Track ON PlaylistTrack.TrackId = Track.TrackId` joins the *Track* table to the intermediate table created by the first join.
* Sometimes the order in which joins are specified and sometimes it doesn't. For the preceding query, the order does not matter. If a query contains only inner joins, the order never matters. If a query contains some left or right joins, the order might matter.
* SQLite may or may not use the order specified in the query. SQLite may decide that it can get the same results with less work if it changes the query order. This is a complicated topic. [See this description of SQLite's internal optimzer for mor information.](https://www.sqlite.org/optoverview.html)

### G. Keys and Relationships
We can finally see why our databases are called *Relational* databases. Each database contains several different tables, but there are relationships between the tables. The relationships are managed via key values that are stored in each table.

#### Keys
Consider the *Customer* table.

In [ ]:
query = """SELECT *
             FROM Customer;"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df.head()

As expected, each record in the *Customer* table has columns like *FirstName*, *LastName*, *City*, etc.  But each record also has a mysterious *CustomerId* number. What does this number mean? Nothing! The *CustomerId* integers were arbitrarily assigned when as each new customer was added to the table. But even though they *mean* nothing, they are still important. The key values are used to identify related records in other tables. Consider the *Invoice* table.

In [ ]:
query = """SELECT *
             FROM Invoice;"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df.head()

FYI, an invoice is what businesses call a bill. Suppose you ran a factory and you ordered some sprockets from *Spacely Sprockets*. You would send a purchase order to *Spacely Sprockets* specifying what sprockets you wanted to purchase, when and to where they should be shipped, and the price you intended to pay for them. *Spacely Sprockets* would ship you the sprockets and then would send you an invoice, listing the products shipped to you and how much money you owed them.

Our *Invoice* table does not have any information about the customers who bought the materials listed on the invoice. But it does have a *CustomerId* column containing an arbitrary-looking integer. For example, invoice #1, which was billed to someone in Germany, has a *CustomerId* of 2. The *Customer* table also has a record with a *CustomerId* of 2, and that customer, named Leonie Köhler, has an address in Stuttgart, Germany. We're using the *CustomerId* values to match records in the *Invoice* table with records in the *Customer* table.

In the *Customer* table, the *CustomerId* column is a *primary key*. It was designated as a primary key when the *Customer* table was created. Primary keys must contain a value (they can't be NULL) and every value must be unique.

In the *Invoice* table, the *CustomerId* column is a *foreign key*. Foreign keys are columns that link to a primary key in another table. Foreign key values do not need to be unique. That makes sense, because if they were required to be unique then customers would only be allowed to have one invoice.

#### Relationships
The relationship between the *Invoice* and *Customer* table is a *many-to-one* relationship. One customer can have many invoices, but each invoice can only have one customer. *many-to-one* relationships occur frequently in relational databases.

It's also possible to have *one-to-one* relationships, where one record corresponds to exactly one record in another table and vice versa. For example, a person is allowed to have exactly one driver's license, and a driver's license lists eactly one driver. We don't see one-to-one relationships as often in relational databases because if there is a one-to-one relationship between two tables, we generally just combine the two tables into a single table.

Finally, the relationship between the *Playlist* and *Track* tables is an example of a *many-to-many* relationship. Each playlist can have several tracks, and each track can be a member of more than one playlist. Many-to-many relationships are more tricky to implement than *many-to-one* or *one-to-one* relationships. Consider the *PlaylistTrack* table.

In [ ]:
query = """SELECT *
             FROM PlaylistTrack;"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df.head()

The *PlaylistTrack* table consists of two foreign key columns. One of the columns, *PlaylistId*, links to the *Playlist* table, and the other column, *TrackId*, links to the *Track* table. Close inspection of this table shows that a single playlist can have many tracks.

In [ ]:
# Playlist 16 has 14 tracks
query = """SELECT *
             FROM PlaylistTrack
            WHERE PlaylistId = 16;"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df

In [ ]:
# Track 52 appears on four different playlists
query = """SELECT *
             FROM PlaylistTrack
            WHERE TrackId = 52;"""
con = sqlite3.connect(chinook_db)
df = pd.read_sql_query(query, con)
con.close()
df

Even though there is a relationship between the *Playlist* and *Track* tables, there is no *TrackId* column on the *Playlist* table and there is no *PlaylistId* on the *Track* table. The two tables are linked only via the *PlaylistTrack* table. This is expected. The *PlaylistTrack* table is called a *joining table*. Joining tables are the standard method for implementing a many-to-many relationship in a relational database.

#### Schema Diagrams
Drawing a diagram of a database can help you understand it. The figure below shows the database schema for the Chinook database.

##### Chinook Database Schema Diagram
![Chinook Database Schema](images/chinook_schema.png)

The diagram was generated automatically by a program called DBSchema. Each box represents a table. The table name is at the top of the box and the columns are listed below. Arrows between boxes show the relationships between different tables.

### H. Exercises 1 - 8

#### Ex. #1
From the Chinnok database, display a table of invoice dates, customer last names, customer first names, and invoice totals. Order the table by invoice total, in descending order with the most expensive invoice first. Limit the size of the table to the ten most expensive invoices.

In [ ]:
# Ex. #1



#### Ex. #2
From the WASNO database, display the team numbers of the teams that are missing from the *teams* table. The results should have a single column, three rows, and be ordered by team number.

Hints:
* This query requires a join, even though the results contain only a single column.
* Use this syntax to check for an empty field: `WHERE colname IS NULL`.

In [ ]:
# Ex. #2



#### Ex. #3
From the Chinnok database, show all tracks on the album by the artist named *The Police*. Your output should have three columns name *Artist*, *Album*, and *Track*. Do not hard code any key values into your query. The only literal value in any `WHERE` clause should be *The Police*. Watch out for case sensitivity. Hint: You need to join three tables for this query.

In [ ]:
# Ex. #3



#### Ex. #4
This exercise has two parts.

Tables can be joined with themselves. Such joins are called self-joins. The *Employee* table in the Chinook database can support a self-join. Write a query that displays the *Employee* table.

In [ ]:
# Ex #4 part 1, display entire Employee table



The *ReportsTo* column is a foreign key that refers to the *Employee*'s table own primary key. It identifies the record in the *Employee* table that holds the information for the employee's manager. (The contents of the *ReportsTo* column appears to be a float because of how Pandas interpreted the column's contents -- it's actually an integer column.)

Display a table with every employee's first name, last name, title, and the first name and last name of their manager. The query should list all employees, even if they don't have a manager.

Hints:
* This query requires a self-join.
* Since the table name appears twice in the join statement, self-joins require that you rename the table twice using aliases. Use *Employees* and *Managers* for your aliases. For example,

  ```sql
  Employee AS Employees INNER/LEFT JOIN Employee AS Managers
  ON Employees. ... = Managers. ...
  ```

  <br/>[You can find another example of a self-join on the *W3 Schools Tutorial*](https://www.w3schools.com/sql/sql_join_self.asp)
* All column names that follow the `SELECT` keyword must be prefaced with the table's alias name. For example, `Employees.FirstName` or `Managers.LastName`.

In [ ]:
# Ex #4 part 2, self-join



#### Ex. #5
What type of join is required for the query in exercise #2 for finding the missing teams? Why? What would happen if you used the other type of join? Why?

In [ ]:
# Ex #5
#
#

#### Ex. #6
How would you change the query in exercise #4, the self-join, to show the manager's title instead of the employee's title.

In [ ]:
# Ex #6
#
#

#### Ex. #7
Inspect the schema diagram for the Chinook database in section II.G.
  1. What do the blue and gold arrows indicate? 
  2. What does the key symbol indicate?
  3. What do the letters on the right side of each row (e.g., 't', 'b', 'd') indicate?
  4. What type of table relationships are displayed in the diagram? How can you tell what type of relationship is displayed?

In [ ]:
# Ex. #7
#
#
#

#### Ex. #8
Imagine that you are creating a database for tracking all of the students and classes at your school. The database should contain all of the data needed to print a class schedule for a student or teacher. A. Describe the database tables, i.e., schema, you would need for this system. Discuss the types of relationships (e.g, many-to-one, many-to-many) between the different tables.
B. Can your schema handle multiple instances of a class? For example, there could be three different sessions of Physics that are taught by two different teachers (one of the teachers teaches two different sessions). If you schema cannot handle this, how would you change it so it could?

Hint: It will help to draw a schema diagram for your database, similar to the diagram in section II.F.

In [ ]:
# Ex #8
#
#

## III. Insert, Update, and Delete Queries
This section explains how to make changes to the data in the database. Changes are accomplished with `INSERT`, `UPDATE`, and `DELETE` queries.

We don't want to mess up the *wasno2020.sqlite* database file. We'll copy the WASNO database into computer memory and use the copy to practie `INSERT`, `UPDATE`, and `DELETE` queries. The next cell copies the databse into memory.

In [ ]:
# Copy WASNO database into memory

# Get connection to database store on disk
wasno_con = sqlite3.connect(wasno_db)
# Create an empty database in computer memory
memcon = sqlite3.connect(":memory:")
# Copy the WASNO database into the memory database.
wasno_con.backup(memcon)

We're going to keep `memcon` open. If close it, we'll lose the database that's stored in memory. [You can read more about storing database in memory and the `sqlite3.Connection.backup` method in the official documentation for the Python Standard Library](https://docs.python.org/3/library/sqlite3.html).

### A. `INSERT` Queries
Let's use an `INSERT` query to add another team into the *Teams* table.

In [ ]:
query = """
    INSERT INTO Teams (team_number, team_name, city, state)
         VALUES ('4613', 'Barker Redbacks', 'Sydney', 'New South Wales');
"""

# Execute the update query
memcon.execute(query)
memcon.commit()

pd.read_sql_query("SELECT * FROM Teams WHERE city = 'Sydney';", memcon)

Insert queries are not hard to understand. We include the table and column names into which we want to insert the data, then we specify the data that we want to insert. The data values must be specified in the same order as the column names.

You can run the preceding cell once with no problems. If you try to run it again, you'll get an error. The reason is that there is a uniquness constraint on the *team_number* column. SQL will not allow duplicate values in this column. Since there is already a team with number 4613 in the *Teams* table the second time we try to run the query, SQL throws an error and does nothing. 

You probably noticed that we used different functions for executing the query. So far, all of our queries have *read information from* the database. The `UPDATE` query is different. It's the first query that *writes information to* the database. The `pandas.read_sql_query()` method will not run queries that write information to the database. We will cover the `.execute()` and `.commit()` methods in the next section.

#### Primary Key Values
Did you notice that SQL inserted the number 38 into the *team_id* column, even though we didn't specify that column? What gives?  Let's look at the table schema to see if we can figure this out.

In [ ]:
pd.read_sql_query("PRAGMA table_info(Teams);", memcon)

The *team_id* column has a *1* in the *pk* field, which means the column was defined as a primary key. In SQLite, INTEGER primary keys cannot contain NULL values. When we try to insert a record the *Teams* table without specifying a value for *team_id*, SQLite chooses an integer to insert into the field. SQLite will choose an integer that does not already appear in that column. This behavior is similar to to other relatinal database programs.

We often need to know what primary key was generated by an `INSERT` statement. We could use a `SELECT` statement retrieve the record we just inserted and get the key, but this is problematic and requires extra work. Fortunately there is a way to get the key value without a second SQL query.

In [ ]:
query = """
    INSERT INTO Teams (team_number, team_name, city, state)
         VALUES ('503', 'Frog Force', 'Novi', 'Michigan');
"""

# Execute the update query
id_val = memcon.execute(query).lastrowid
print("New team_id value:", id_val)
memcon.commit()

pd.read_sql_query(f"SELECT * FROM Teams WHERE team_id = {id_val};", memcon)

The `.execute()` returns an object with a `.lastrowid` attribute. This attribute contains the primary key for the row that was inserted.

#### Skipping the Column Names
We can skip the column names if we insert data into every column, in the order that the columns were defined when the table was created.

In [ ]:
query = """
    INSERT INTO Teams 
         VALUES (NULL, '1816', 'The Green Machine', 'Edina', 'Minnesota', 'USA', 2006, NULL);
"""

# Execute the update query
memcon.execute(query)
memcon.commit()

pd.read_sql_query("SELECT * FROM Teams WHERE team_number = '1816';", memcon)

Even though we entered `NULL` for the *team_id* column, SQLite created a new integer primary key for the record.

[Refer to the *W3 Schools SQLA Tutoral* for more information on `INSERT` queries.](https://www.w3schools.com/Sql/sql_insert.asp)

### B. UPDATE Queries
The *team_name* and other data is missing for FRC 7461. Their name is *Sushi Squad*. Let's fix that.

In [ ]:
# Before the update query
select_query = """
    SELECT *
      FROM teams
     WHERE team_number = '7461';
"""
display(pd.read_sql_query(select_query, memcon))

# This query changes an existing record
print("Running the Update Query")
update_query = """
    UPDATE teams
       SET team_name = 'Sushi Squad',
           city = 'Redmond',
           state = 'Washington'
     WHERE team_number = '7461';
"""
# Execute the update query
memcon.execute(update_query)
memcon.commit()

# Check the results
display(pd.read_sql_query(select_query, memcon))

The `UPDATE` query changes records that already exist in a database table. The `WHERE` clause is extremely important. If we had omitted it, all team names would have been changed to 'Sushi Squad'. We would be making things very hard for the match announcer if every team in the match had the same name.

Update queries cannot can't add new records to a table. Depending on how you write the query, SQL will either do nothing, or completely mess up your data.
```sql
-- Does nothing because there is no team '42397'. 
UPDATE teams
   SET team_name = 'Spacely Sprocketeers',
       city = 'Orbit City'
 WHERE team_number = '42397';
 
-- Every team in the database is now the Spacely Sprocketeers
UPDATE teams
   SET team_name = 'Spacely Sprocketeers',
       city = 'Orbit City'
       team_number = '42397';
```
[Refer to the *W3 SQL Tutorial* for additional information on SQL Update Queries.](https://www.w3schools.com/Sql/sql_update.asp)

### C. `DELETE` Queries

In [ ]:
select_query = """
    SELECT *
      FROM Teams
     WHERE team_number = '1318';
"""
display(pd.read_sql_query(select_query, memcon))

# Delete Query
print("Running the DELETE query!")
del_query = """
    DELETE FROM Teams 
         WHERE team_number = '1318';
"""

# Execute the update query
memcon.execute(del_query)
memcon.commit()

pd.read_sql_query(select_query, memcon)

`DELETE` queries are simple, perhaps a bit too simple. Specify the table and use a `WHERE` clause to identify the records that will be deleted, and you are in business.

But what happens if you mess up or forget the `WHERE` clause? You might just erase some or all of your records.

In [ ]:
# Bad DELETE Query
select_query = 'SELECT COUNT(*) AS "Records in Measures Table" FROM measures;'
display(pd.read_sql_query(select_query, memcon))

# Run DELETE Query
print('Running the DELETE query...')
bad_query = 'DELETE FROM Measures;'
memcon.execute(bad_query)
memcon.commit()

display(pd.read_sql_query(select_query, memcon))

Whoops! It's too bad about the *Measures* table.

### D. Parameterized Queries
It's a common and useful practice to wrap SQL queries in a Python function that accepts parameters. Suppose we wanted `get_team()` and `add_team()` functions in our program that would get a team from or add a team to the *Teams* table. We could write the functions like this:

In [ ]:
def get_team_risky(con, team_number):
    query = "SELECT * FROM Teams WHERE team_number = '" + team_number + "';"
    return pd.read_sql_query(query, con)

def add_team_risky(con, team_number, team_name):
    query = ("INSERT INTO Teams (team_number, team_name)" +
             "VALUES ('" + team_number + "', '" + team_name + "');")
    con.execute(query)
    con.commit()
    
add_team_risky(memcon, "1902", "Exploding Bacon")
get_team_risky(memcon, "1902")

The functions work as written, but manually constructing the queries from variables is tedious. But's that is not the biggest problem. What if someone passed the following parameter to `.get_team()`?
```python
evil_team_number = """
    1318';
    DELETE FROM Teams;-- 
"""

get_team_risky(memcon, evil_team_number)
```

[`evil_team_number` is what's known as a SQL injection attack.](https://en.wikipedia.org/wiki/SQL_injection) It's an example of why it's dangerous to manualy insert strings into SQL queries. 

#### Figure 3, SQL Injection
![XKCD Comic](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)<br/>
From [xkcd.com](https://xkcd.com/). Creative Commons Attribution-NonCommercial 2.5 License.

The sqlite3 package supports parameterized queries that reduce the risk of SQL injection attacks.

In [ ]:
def get_team(con, team_number):
    query = "SELECT * FROM Teams WHERE team_number = ?;"
    return con.execute(query, [team_number]).fetchone()

def add_team(con, team_number, team_name):
    query = "INSERT INTO Teams (team_number, team_name) VALUES (?, ?);"
    con.execute(query, (team_number, team_name))
    con.commit()
    
add_team(memcon, "2834", "Bionic Black Hawks")
get_team(memcon, "2834")

To use parameterized queries, put a question mark, '?', wherever you want to insert the value of a parameter. When you call the `.execute()` method, add a second argument that contains the parameter values you want to insert into the query. The second argument to `.execute()` must be a sequence object, like a tuple or list. Parameterized queries provide several benefits.
* No need to manually assemble query strings.
* Protection for SQL injection.
* Autmatically handles quotation markes. Adds quotation marks for string datatypes and omits them for numeric types.

In addition to the question mark style of parameterized queries, the sqlite3 package also supports a named style.

In [ ]:
def get_team(con, team_number):
    query = "SELECT * FROM Teams WHERE team_number = :team_number;"
    return con.execute(query, {"team_number": team_number}).fetchone()

def add_team(con, team_number, team_name):
    query = """
        INSERT INTO Teams (team_number, team_name)
             VALUES (:team_number, :team_name);
    """
    con.execute(query, {"team_name": team_name, "team_number": team_number})
    con.commit()
    
add_team(memcon, "1311", "Kell Robotics")
print(get_team(memcon, "1311"))
memcon.close()

### E. Exercises 9 - 11
Use the `ccon` connection object for exercises 9 - 11.

In [ ]:
# Use in-memory copy of Chinook database
chinook_con = sqlite3.connect(chinook_db)
ccon = sqlite3.connect(":memory:")
chinook_con.backup(ccon)

#### Ex. #9
Use an INSERT query to add an artist, any artist, to the  *Artist* table in the Chinook database. Use a SELECT query to verify the artist was added.

In [ ]:
# Exercise #9
query = """
-- Write your INSERT query here
"""

ccon.execute(query, ccon)
ccon.commit()

# Execute your SELECT Query here



#### Ex. #10
Use an UPDATE query that changes the names of all albumns containing the word "Rock". The new albumn name should be "Robot Boogie". The cell will run with no errors if the UPDATE query is correct.

In [ ]:
# Exercise #10
query = """
    -- Write your UPDATE query here
    
"""
# This code runs and tests your query. Do not alter.
ccon.execute(query)
ccon.commit()
test_query = """
    SELECT COUNT(*)
      FROM Album
     WHERE Title = 'Robot Boogie';
"""
assert ccon.execute(test_query).fetchone()[0] == 7

#### Ex. #11
Write a DELETE query that deletes all albums from the *Album* table that contain the word "Song".

In [ ]:
query = """
    -- Write your DELETE query here
"""


# This code tests your query. Do not alter.
ccon.execute(query)
ccon.commit()
test_query = """
    SELECT COUNT(*)
      FROM Album;
"""
assert ccon.execute(test_query).fetchone()[0] == 344

## IV. Running Queries
This notebook introduces two techniques for running SQL queries from Python.
* The `sqlite3.connection.execute()` method
* *Sqlite3's* `cursor` object.

### A. The `connection.execute()` Method
For queries that write to the database and do not return any data, you must use the `execute()` method from the `sqlite3` package. Here is an example:
```python
# Import the sqlite3 package
import sqlite3

# Get a connection object to the database file
con = sqlite3.connect("database-file.sqlite3")

# Create and execute the query
query """UPDATE some_table
            SET some_column = 'some_value'
            WHERE some_other_column = 'some_other_value';"""
con.execute(query)

# Commit the changes
con.commit()
con.close()
```

This is not too hard. We just run the `execute()` method on the connection object, passing the SQL query as an argument.

The only mysterious part of the code snippet is the line `con.commit()`. If we were to omit this line, the changes would not be saved to the database. It's as if the `commit()` function is really the `simon_says()` function. If we don't say *Simon Says...*, nothing actually happens.

The `commit()` function seems superflous but it exists for a very good reason. Suppose you have a savings and a checking account at a local bank. The bank stores account records in a SQL database. They have an *account_balances* table that contains the account balance of every account.
* Now suppose that you try to transfer &dollar;100 from your checking account to your savings account.
* This transfer requires two `UPDATE` queries. The first `UPDATE` query subtracts &dollar; 100 from your checking account balance. The second query adds &dollar; 100 to your savings account balance.
* Suppose the first query, which decrements your checking account balance by &dollar; 100, runs just fine. But then there is a glitch, and the second `UPDATE` query throws an error and doesn't run.
* You have just lost &dollar; 100. &dollar; 100 was subtracted from your checking account, but there was no corresponding increase in your savings account balance. That's bad.

We want both queries to run with no errors. But if one query fails, it would be best if none of the queries run. Most of us would prefer that the transfer not happen at all over having &dollar;100 mysteriously dissapear from our checking account. The `commit()` function exists for this purpose. Consider the following code.

```python
subtract_query = """
    UPDATE balances
       SET balance = balance - 100
     WHERE user_id = '9876543'
       AND account = 'checking';"""
       
add_query = """
    UPDATE balances
       SET balance = balance + 100
     WHERE user_id = '9876543'
       AND account = 'savings';"""
       
con.execute(subtract_query)
con.execute(add_query)
con.commit()
```

The `con.commit()` method allows us to group SQL queries together. If either the `subtract_query` or `add_query` fail, execution will stop before the `con.commit()` method is run, and no changes will be saved to the database. A consequence of this is that you need to run `con.commit()` after you run one or more queries that update the database, or your updates will not be saved.

A group of SQL statments that must be run as a group, with either all or none of the SQL statements being executed, is called a *transaction*. It is possible to configure *SQLite* to automatically commit every SQL query without calling `con.commit()`, but we're not going to do that.

[Refer to the Python *sqlite3* documentatoin for additional information on the `.commit()` and `.rollback()` methods.](https://docs.python.org/3/library/sqlite3.html#connection-objects)

### B. Using Cursors
So far we've used two different techniques to run SQL queries on our *SQLite* database. We used *Pandas* `.read_sql_query()` for queries that return information and the *sqlite3* package's `.execute()` methods for queries that don't return any information. You may be wondering ... would we ever use the `.execute()` method for queries that do return information? The answer is yes, absolutely! But first, we need to learn about cursors.

#### Cursor Etymology
You already know about two kinds of cursors. Your computer mouse's cursor indicates the position on the screen that will be activated if you click the mouse's button. Word processors, text editors, and CLIs use a cursor to indicate the position at which text will be entered if you type characters on your keyboard. The word cursor comes from the Latin word *cursor*, which means runner or courier. Before computers were commonplace, the word cursor was [frequently used to describe the transparent slide on a slide rule](https://www.math.utah.edu/~alfeld/sliderules/) (see figure below). A slide rule cursor moves back and forth along the slide rule and has a vertical hariline that helps the users line up different scales on the slide rule. 

![Slide Rule](images/sliderule.jpg)

#### Database Cursors
In database programming, the word *cursor* has a special meaning. It refers to a software object that marks the current position within a set of database records that have been returned from a query. But enough talk -- let's look at an example.

In [ ]:
# Using a cursor object
query = """SELECT team_number, team_name, city, state, year_founded FROM teams LIMIT 6;"""
con = sqlite3.connect(wasno_db)
cursor = con.cursor()
cursor.execute(query)
for record in cursor:
    print(record)

Here's what happened.
1. First, we retrieved a cursor object by calling the `.cursor()` method of the *sqlite3* connection (`con`) object.
2. We executed the query by calling the cursor's `.execute()` method.
3. Finally, we iterated over the cursor and printed the results. Our `for` loop extracted one record at a time from the query results.

Each record is a Python tuple that contains the record's fields. The fields are in the same order as the column order specified in the SQL query. If we had used the asterisk instead of explicitly naming the columns, the field order would have matched the order of the columns in the table's `CREATE` query. Individual fields can be extracted from the tuple using indexing notation:
```python
# Using Index Notation to Retrieve Fields

# Get the team number
team_number = record[0]
# Get the city
team_number = record[2]
```

There is an interesting thing about cursors. Suppose we want to loop over the query results again:

In [ ]:
# Iterating again
for record in cursor:
    print(record)
con.close()

No records were returned when we tried to iterate over the cursor a second time. That's because the cursor object does not contain the query results. Instead, it points to the query data in the database. Like a text cursor that indicates your current position within a document, the database cursor indicates the position within the database of the data we want to retrieve.

Before the `for` loop, the cursor points to the first record of the query results. The first record is not retrieved from the database until we enter the `for` loop for the first time. On subsequent iterations of the `for` loop, the data in the `record` variable is overwritten with a new tuple containing the next record. After the final record is retrieved and the `for` loop is finished, the cursor is exhausted and points to the end of the query results.

Other than re-running the query, there is no way to make the cursor move backwards and point to an earlier record or the first record. Either you save the record within the body of a `for` loop, or the record is lost forever (or until you re-run the query).

#### Benefits of Using Cursors
Why would the cursor object behave like this? Why can't we just get a data-structure that contains all the records and peruse it at our convenience, in whatever order we choose? It turns out there is a good reason the cursor behaves the way it does. The following code selects all records from the *measures* table using *Pandas* and a cursor object and compares the amount of memory used. It uses the [`sys.getsizeof()` function from the *Python Standard Library*](https://docs.python.org/3/library/sys.html#sys.getsizeof) to get the amount of memory in bytes required to store an object.

In [ ]:
# Using a cursor object
query = """SELECT * FROM measures;"""

# Run query using Pandas
con = sqlite3.connect(wasno_db)
measures = pd.read_sql_query(query, con)
# Get amount of memory consumed by
# measures dataframe 
df_bytes = sys.getsizeof(measures)

# Run query using a cursor object
cursor = con.cursor()
cursor.execute(query)
# Check amount of memory used by cursor and record at each
# iteration of for loop. Keep track of maximum size.
max_cursor_bytes = sys.getsizeof(cursor)
for record in cursor:
    max_cursor_bytes = max(max_cursor_bytes,
                           sys.getsizeof(cursor) + sys.getsizeof(record))
max_cursor_bytes = max(max_cursor_bytes,
                           sys.getsizeof(cursor) + sys.getsizeof(record))

# Display results
print(f"Bytes used by cursor: {max_cursor_bytes:,}")
print(f"Bytes used by dataframe: {df_bytes:,}")
print(f"Ratio of dataframe to cursor memory used: {int(df_bytes / max_cursor_bytes):,}")
con.close()

Retrieving all of the data using *Pandas* uses over 4,000 times as much memory as using a cursor object! When we use the cursor object to retrieve data, we never hold more than one record in memory at a time, which means the cursor object uses minimal memory.

The entire *measures* table requires about 1.5 Mb of memory, which is still quite small. Hundreds of megabytes to a Gigabyte are usually not a problem for 2021-era computers. The IRS's scouting data has always been small enough to fit in memory, so deciding between *Pandas* or *sqlite3's* cursor object is a matter of personal preference.

Someday you might work with *really* big data. SQLite databases can contain many terabytes of information (max theoretical size of a *SQLite* database is 140 TB), so a table could easily take up dozens or hundreds of gigabytes. Such a table cannot be read into memory all at one time - the `pandas.read_sql_query()` method would crash. But with a cursor object, you could scan through the entire table one record at a time, analyze each record, and save the results of your analysis back to the database or to disk.

#### Getting Column Names from the Cursor

##### Using the `.description` Attribute
Consider the following query.

In [ ]:
# Using a cursor object
query = """SELECT * FROM measures LIMIT 1;"""
con = sqlite3.connect(wasno_db)
cursor = con.cursor()
cursor.execute(query)
for record in cursor:
    print(record)
con.close()

We used an asterisk in the `SELECT` statement to get all columns from the *measures* table. The *measures* table has a lot of columns, and it's not easy to determine which columns are which. The order of the values within the tuple is the same order in which the columns were listed in the `CREATE` SQL statement that was used to create the *measures* table. We could run another SQL query to get the column names (see section III), but the *sqlite3* cursor object provides an easier way. Just access the `.description` attribute on the cursor object.

In [ ]:
# Get column names
cursor.description

The `.description` attribute returns a tuple of tuples. Each inner tuple represents a column from the query results. The inner tuples will always have seven elements, with the first element containing the column name and the remaining six elements always containing the `None` object. The reason for the `.description` attribute's odd structure is that it needs to be compatible with other (non-*SQLite*) database systems. Other database systems presumeably return information in addition to the column name.

The `.description` attribute can easily be converted to a simple list of column names with a list comprehension.

In [ ]:
# Getting a Simple List of Column Namesf
col_names = [col[0] for col in cursor.description]
print(col_names)

##### Using Row Factories
Row factories are a sqlite3 alternative to the `.description` attribute. Setting the `.row_factory` property of the `sqlite3.connection` object to `sqlite3.Row` causes the cursors to return named tuples. With named tuples, we can extract values from fields using `row['coname']` syntax.

In [ ]:
# Accessing Fields with Column Names Using a Row Factory
con_colnames = sqlite3.connect(wasno_db)
con_colnames.row_factory = sqlite3.Row

# Run the query
query = """SELECT * FROM measures;"""
cursor = con_colnames.execute(query)

# Retrieve the first results row
row = cursor.fetchone()

print("Each row works like a Python dictionary, with column names as keys.")
print()
print("Column Names")
print(row.keys())
print()
print("Example of retrieving fields with column names")
print(f"Team: {row['team']}, Match: {row['match']}, Date: {row['date']}")
con_colnames.close()

#### Cursor Shortcuts
So far, we've been explicitly creating cursor objects with the statement `cursor = con.cursor`. The *sqlite3* package provides a shortcut. If we call the `.execute()` method directly from the connection object, the `.execute()` method will execute the query and return a cursor object. This allows us to eleminate the statement that creates the cursor. An example is provided below.

In [ ]:
# Cursor Shortcut - Calling Execute from Connection Object
# This style eliminates one statement
query = """SELECT team_number, team_name, city, state, year_founded FROM teams LIMIT 6;"""
con = sqlite3.connect(wasno_db)
cursor = con.execute(query)
for record in cursor:
    print(record)
con.close()

We can use an even shorter sytax if desired.

In [ ]:
# Cursor Shortcut - Calling Execute from Connection Object
# This style eliminates two statements!
query = """SELECT team_number, team_name, city, state, year_founded FROM teams LIMIT 6;"""
con = sqlite3.connect(wasno_db)
for record in con.execute(query):
    print(record)
con.close()

The final example is short and easy to understand. Which style you use is a matter of personal preference. Note that in the final example, it's not possiblel to get the column names from the cursor object because the cursor object is never saved to its own variable.

#### Cursor Methods
We've seen how we can iterate over a sqlite3 cursor object with a `for` loop or a loop comprehension. We can also use one of the cursor object's methods to retrieve data. First, let's run a query and get a cursor object.

In [ ]:
query = """
    SELECT Name, Title
      FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
  ORDER BY Name;
"""
con = sqlite3.connect(chinook_db)
cursor = con.cursor()
cursor.execute(query)

##### `.fetchone()`
The `.fetchone()` method retreives the next record from the cursor object. It's especially useful when retrieving a single record from the database, like `SELECT COUNT(*) FROM Some_Table;`, but can be used with multi-record results too.

In [ ]:
# Use .fetchone() to fetch one record at a time
# Fetch a record
print(cursor.fetchone())

# Fetch another record
tuple(cursor.fetchone())

##### `.fetchmany()`
The `.fetchmany()` method retreives however many records are specified in it's argument.

In [ ]:
# Use .fetchmany()` to get a specified number of records
print(cursor.fetchmany(3))

##### `.fetchall()`
The `.fetchall()` method gets all remaining records from the cursor as a Python list. Be careful with this method. It's possible to return so many records that your computer runs out of memory.

In [ ]:
# Use .fetchall() to get all remaining records
remaining_albums = cursor.fetchall()
cursor.close()
print("Number of records returned:", len(remaining_albums))
print("\nFirst five records:")
print(remaining_albums[:5])

### C. Exercises 12 - 16

#### Ex. #12
Using a cursor object and a `for` loop, build a list of all team numbers that completed ten or more outer goals in any match during teleop. Ensure there are no repeated values in the list.

Hints:
* Use the *task*, *phase*, and *successes* columns from the *Measures* table.
* The task name for outer goals is 'launchOuter' and the phase name for telop is 'teleop'.
* Your list shold contain seven teams.

In [ ]:
# Ex #12



#### Ex. #13
Use the same query to generate the same list of teams as in exercise #12. But instead of using a `for` loop, use a list comprehension.

In [ ]:
# Ex. #13



#### Ex. #14
Add a join to the query you used in exercises 12 and 13 so you can generate a list of tuples. Each tuple will contain both the team number and team name for all teams that scored 10 or more outer goals in teleop during a single match

In [ ]:
# Ex. #14



#### Ex. #15
Use the `.backup()` method of sqlite3's `Connection` object to copy the Chinook database into memory. Get a connection to in-memory instance of the Chinook database for use in exercise #16, #18, and #20.

In [ ]:
# Ex. #16



#### Ex. #16
Use an INSERT query to insert yourself into the *Employee* table of the Chinook music store.
* Use the connection to the in-memory database tht you created in exercise #15.
* Your title should be Chief Technical Officer and you should report directly to the general manager.
* Use the `.lastrowid` attribute of the `cursor` object to get the primary key for the new record.
* Write a parameterized SELECT query that uses the value from the `.lastrowid` attribute to display the record you just entered into the *Employee* table.

* Identify the primary keys in the Invoice table
* Multi-table join using invoices
* Execute Many
* fetchmany returns None

## V. Exercises 17 - 20

#### Ex. #17
How many foreign key fields are in the Chinook database's *Track* table? What are they?

In [ ]:
# Ex. #17
#
#

#### Ex. #18
Using the Chinook database's *Track* table as one of the tables, construct a SELECT query with a multi-table join.
* Use the join to replace every foreign key with a descriptive column from the linked table.
* The result should be a table of all tracks (and no foreign keys).
* Order the table by the name of the track.

In [ ]:
# Ex. # 18



#### Ex. #18
Write a function that uses a parameterized INSERT query.
* Use the connection to the in-memory Chinook database that you created in exercise #15, so you don't alter the Chinook database file.
* The function should accept customer data as arguments and insert a new record into the in-memory Chinook database's *Customer* table.
* The function should return the *CustomerId* of the inserted record.

In [ ]:
# Ex #18



#### Ex. #19
What does the `.fetchmany()` method return if there are no records left in the cursor object? What do the `.fetchmany()` and `.fetchall()` methods return if no records are left?

Don't guess. Look up the answers in the [sqlite3 package's documentation](https://docs.python.org/3/library/sqlite3.html).

In [ ]:
# Ex #19
#
#

#### Ex. #20
Use sqlite3's `.executemany()` function to enter the following artists into the Chinook database's *Artist* table. These are real artists.
* Ayron Jones
* The Blacktones
* Deep Sea Diver
* Pickwick
* Tacocat

Instructions:
* Use the connection to the in-memory Chinook datbase you created in exercise #15.
* The `.executemany()` function has not been covered in this notebook. Figure out how to use it be reading about it in the [sqlite3 package's documentation](https://docs.python.org/3/library/sqlite3.html).
* You must include a comma when creating length-one tuples, e.g., `my_tuple = (1, )`.

In [ ]:
# Ex. #20



## VI. Save Your Work
Once you have completed the exercises, save a copy of the notebook outside of the git repository (outside of the *pyclass_frc* folder). Include your name in the file name. Send the notebook file to another student to check your answers.

## VII. Concept and Terminology Review
You should be able to define the following terms or describe the concept. 
* `LEFT JOIN`
* `INNER JOIN`
* `ON`
* RIGHT JOIN
* OUTER JOIN
* Cross Join
* Cartesian Product
* Self Join
* Primary Key
* Foreign Key
* One to One
* Many to One
* Many to Many
* Schema
* INSERT
* UPDATE
* DELETE
* Parameterized Queries
* Question mark style Parameterized Query
* Named style parameterized query
* SQL Injection Attack
* `.connect()`
* `.execute()`
* `.executemany()`
* `.commit()`
* `.cursor()`
* `.lastrowid`
* `.row_factory`
* `.fetchone()`
* `.fetchmany()`
* `.fetchall()`


[Table of Contents](../../index.ipynb)